In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [2]:
!mkdir -p data
!wget -qq -c https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O data/yellow_tripdata_2023-01.parquet
!wget -qq -c https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -O data/yellow_tripdata_2023-02.parquet

In [3]:
df1 = pd.read_parquet("data/yellow_tripdata_2023-01.parquet", engine="pyarrow")

In [4]:
print(df1.columns)
print("#columns: ", len(df1.columns))

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
#columns:  19


**`Answer`**: 19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [5]:
df1["duration"] = df1["tpep_dropoff_datetime"] - df1["tpep_pickup_datetime"]
df1["duration"] = df1["duration"].dt.total_seconds() / 60

In [6]:
print(f"std(duration): {df1.duration.std():.2f}")

std(duration): 42.59


**`Answer`**: 42.59

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [7]:
n_before = len(df1)
print(n_before)
df1 = df1[(df1["duration"] >=1) & (df1["duration"] <= 60)].copy()
n_after = len(df1)

print(f"Remaining Samples: {float(n_after/n_before) * 100.0}")


3066766
Remaining Samples: 98.1220282212598


**`Answer`**: 98%

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [8]:
categorical = ['PULocationID', 'DOLocationID']

In [9]:
df1[categorical] = df1[categorical].astype(str)
train_dicts = df1[categorical].to_dict(orient="records")
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [10]:
print(f"#columns: {X_train.shape[1]}")

#columns: 515


**`Answer`**: 515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [11]:
# Data preparation
target = "duration"
y_train = df1[target].values

In [12]:
# Model training
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [13]:
# Predictions
y_pred = model.predict(X_train)
print(f"RMSE(X_train): {np.sqrt(mean_squared_error(y_train, y_pred)):.6f}")

RMSE(X_train): 7.649262


**`Answer`**: 7.64

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [14]:
# Data Pre-processing
df2 = pd.read_parquet("data/yellow_tripdata_2023-02.parquet", engine="pyarrow")

df2["duration"] = df2["tpep_dropoff_datetime"] - df2["tpep_pickup_datetime"]
df2["duration"] = df2["duration"].dt.total_seconds() / 60
df2 = df2[(df2["duration"] >=1) & (df2["duration"] <= 60)].copy()

df2[categorical] = df2[categorical].astype(str)
val_dicts = df2[categorical].to_dict(orient="records")
X_val = dv.transform(val_dicts)
y_val = df2[target].values

In [18]:
# Model Evaluation
y_val_pred = model.predict(X_val)
print(f"RMSE(Val): {np.sqrt(mean_squared_error(y_val, y_val_pred)):.6f}")

RMSE(Val): 7.811818


**`Answer`**: 7.81